#google cloud project key 인증



In [1]:
!gcloud auth activate-service-account --key-file "siliconwildcat.json"

Activated service account credentials for: [service-587317845756@siliconwildcat.iam.gserviceaccount.com]


#필수 라이브러리 및 함수 불러오기

실행에 필요한 라이브러리 및 함수를 불러옵니다.

이 과정은 약 10분 정도 소요될 수 있습니다.

In [2]:
!git clone https://github.com/sce-tts/TTS.git -b sce-tts

Cloning into 'TTS'...
remote: Enumerating objects: 15717, done.
remote: Total 15717 (delta 0), reused 0 (delta 0), pack-reused 15717
Receiving objects: 100% (15717/15717), 125.93 MiB | 21.75 MiB/s, done.
Resolving deltas: 100% (11278/11278), done.


In [3]:
import os
import sys
from pathlib import Path

In [4]:
!git clone https://github.com/sce-tts/g2pK.git

Cloning into 'g2pK'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 110 (delta 52), reused 102 (delta 46), pack-reused 0
Receiving objects: 100% (110/110), 53.79 KiB | 1.38 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [8]:
!gsutil cp -r gs://siliconwildcat_poc_bucket/tts_infer/TTS /content

Caught CTRL-C (signal 2) - exiting
^C


In [5]:
%cd /content/TTS
!pip install -q --no-cache-dir -e .
%cd /content/g2pK
!pip install -q --no-cache-dir "konlpy" "jamo" "nltk" "python-mecab-ko"
!pip install -q --no-cache-dir -e .

/content/TTS
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 69 kB 5.0 MB/s 
     |████████████████████████████████| 1.3 MB 9.1 MB/s 
     |████████████████████████████████| 124 kB 78.0 MB/s 
     |████████████████████████████████| 228 kB 72.7 MB/s 
     |████████████████████████████████| 71 kB 64.1 MB/s 
     |████████████████████████████████| 183 kB 74.7 MB/s 
     |████████████████████████████████| 3.2 MB 69.6 MB/s 
     |████████████████████████████████| 20.1 MB 1.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 25.3 MB 1.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

In [6]:
%cd /content/g2pK
import g2pk
g2p = g2pk.G2p()

/content/g2pK
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [7]:
%cd /content/TTS
import re
import sys
from unicodedata import normalize
import IPython

from TTS.utils.synthesizer import Synthesizer

def normalize_text(text):
    text = text.strip()

    for c in ",;:":
        text = text.replace(c, ".")
    text = remove_duplicated_punctuations(text)

    text = jamo_text(text)

    text = g2p.idioms(text)
    text = g2pk.english.convert_eng(text, g2p.cmu)
    text = g2pk.utils.annotate(text, g2p.mecab)
    text = g2pk.numerals.convert_num(text)
    text = re.sub("/[PJEB]", "", text)

    text = alphabet_text(text)

    # remove unreadable characters
    text = normalize("NFD", text)
    text = "".join(c for c in text if c in symbols)
    text = normalize("NFC", text)

    text = text.strip()
    if len(text) == 0:
        return ""

    # only single punctuation
    if text in '.!?':
        return punctuation_text(text)

    # append punctuation if there is no punctuation at the end of the text
    if text[-1] not in '.!?':
        text += '.'

    return text


def remove_duplicated_punctuations(text):
    text = re.sub(r"[.?!]+\?", "?", text)
    text = re.sub(r"[.?!]+!", "!", text)
    text = re.sub(r"[.?!]+\.", ".", text)
    return text


def split_text(text):
    text = remove_duplicated_punctuations(text)

    texts = []
    for subtext in re.findall(r'[^.!?\n]*[.!?\n]', text):
        texts.append(subtext.strip())

    return texts


def alphabet_text(text):
    text = re.sub(r"(a|A)", "에이", text)
    text = re.sub(r"(b|B)", "비", text)
    text = re.sub(r"(c|C)", "씨", text)
    text = re.sub(r"(d|D)", "디", text)
    text = re.sub(r"(e|E)", "이", text)
    text = re.sub(r"(f|F)", "에프", text)
    text = re.sub(r"(g|G)", "쥐", text)
    text = re.sub(r"(h|H)", "에이치", text)
    text = re.sub(r"(i|I)", "아이", text)
    text = re.sub(r"(j|J)", "제이", text)
    text = re.sub(r"(k|K)", "케이", text)
    text = re.sub(r"(l|L)", "엘", text)
    text = re.sub(r"(m|M)", "엠", text)
    text = re.sub(r"(n|N)", "엔", text)
    text = re.sub(r"(o|O)", "오", text)
    text = re.sub(r"(p|P)", "피", text)
    text = re.sub(r"(q|Q)", "큐", text)
    text = re.sub(r"(r|R)", "알", text)
    text = re.sub(r"(s|S)", "에스", text)
    text = re.sub(r"(t|T)", "티", text)
    text = re.sub(r"(u|U)", "유", text)
    text = re.sub(r"(v|V)", "브이", text)
    text = re.sub(r"(w|W)", "더블유", text)
    text = re.sub(r"(x|X)", "엑스", text)
    text = re.sub(r"(y|Y)", "와이", text)
    text = re.sub(r"(z|Z)", "지", text)

    return text


def punctuation_text(text):
    # 문장부호
    text = re.sub(r"!", "느낌표", text)
    text = re.sub(r"\?", "물음표", text)
    text = re.sub(r"\.", "마침표", text)

    return text


def jamo_text(text):
    # 기본 자모음
    text = re.sub(r"ㄱ", "기역", text)
    text = re.sub(r"ㄴ", "니은", text)
    text = re.sub(r"ㄷ", "디귿", text)
    text = re.sub(r"ㄹ", "리을", text)
    text = re.sub(r"ㅁ", "미음", text)
    text = re.sub(r"ㅂ", "비읍", text)
    text = re.sub(r"ㅅ", "시옷", text)
    text = re.sub(r"ㅇ", "이응", text)
    text = re.sub(r"ㅈ", "지읒", text)
    text = re.sub(r"ㅊ", "치읓", text)
    text = re.sub(r"ㅋ", "키읔", text)
    text = re.sub(r"ㅌ", "티읕", text)
    text = re.sub(r"ㅍ", "피읖", text)
    text = re.sub(r"ㅎ", "히읗", text)
    text = re.sub(r"ㄲ", "쌍기역", text)
    text = re.sub(r"ㄸ", "쌍디귿", text)
    text = re.sub(r"ㅃ", "쌍비읍", text)
    text = re.sub(r"ㅆ", "쌍시옷", text)
    text = re.sub(r"ㅉ", "쌍지읒", text)
    text = re.sub(r"ㄳ", "기역시옷", text)
    text = re.sub(r"ㄵ", "니은지읒", text)
    text = re.sub(r"ㄶ", "니은히읗", text)
    text = re.sub(r"ㄺ", "리을기역", text)
    text = re.sub(r"ㄻ", "리을미음", text)
    text = re.sub(r"ㄼ", "리을비읍", text)
    text = re.sub(r"ㄽ", "리을시옷", text)
    text = re.sub(r"ㄾ", "리을티읕", text)
    text = re.sub(r"ㄿ", "리을피읍", text)
    text = re.sub(r"ㅀ", "리을히읗", text)
    text = re.sub(r"ㅄ", "비읍시옷", text)
    text = re.sub(r"ㅏ", "아", text)
    text = re.sub(r"ㅑ", "야", text)
    text = re.sub(r"ㅓ", "어", text)
    text = re.sub(r"ㅕ", "여", text)
    text = re.sub(r"ㅗ", "오", text)
    text = re.sub(r"ㅛ", "요", text)
    text = re.sub(r"ㅜ", "우", text)
    text = re.sub(r"ㅠ", "유", text)
    text = re.sub(r"ㅡ", "으", text)
    text = re.sub(r"ㅣ", "이", text)
    text = re.sub(r"ㅐ", "애", text)
    text = re.sub(r"ㅒ", "얘", text)
    text = re.sub(r"ㅔ", "에", text)
    text = re.sub(r"ㅖ", "예", text)
    text = re.sub(r"ㅘ", "와", text)
    text = re.sub(r"ㅙ", "왜", text)
    text = re.sub(r"ㅚ", "외", text)
    text = re.sub(r"ㅝ", "워", text)
    text = re.sub(r"ㅞ", "웨", text)
    text = re.sub(r"ㅟ", "위", text)
    text = re.sub(r"ㅢ", "의", text)

    return text


def normalize_multiline_text(long_text):
    texts = split_text(long_text)
    normalized_texts = [normalize_text(text).strip() for text in texts]
    return [text for text in normalized_texts if len(text) > 0]

def synthesize(text):
    wavs = synthesizer.tts(text, None, None)
    return wavs

/content/TTS


#학습한 모델 불러오기

학습한 Glow-TTS와 HiFi-GAN 모델을 불러옵니다.

만약 다른 체크포인트에서 불러오시려면 아래 코드에서 경로를 아래와 같이 적절하게 수정합니다.

```python
synthesizer = Synthesizer(
    "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/glowtts-v2-May-31-2021_08+17AM-d897f2e/best_model.pth.tar",
    "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/glowtts-v2-May-31-2021_08+17AM-d897f2e/config.json",
    None,
    "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/hifigan-v2-May-31-2021_08+26AM-d897f2e/checkpoint_300000.pth.tar",
    "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/hifigan-v2-May-31-2021_08+26AM-d897f2e/config.json",
    None,
    None,
    False,
)
```

In [8]:
%cd /content/

/content


In [9]:
!gsutil cp -r gs://siliconwildcat_poc_bucket/tts_infer/glowtts-v2 /content

Copying gs://siliconwildcat_poc_bucket/tts_infer/glowtts-v2/best_model.pth.tar...
Copying gs://siliconwildcat_poc_bucket/tts_infer/glowtts-v2/config.json...
Copying gs://siliconwildcat_poc_bucket/tts_infer/glowtts-v2/scale_stats.npy...
Copying gs://siliconwildcat_poc_bucket/tts_infer/glowtts-v2/scale_stats_new.npy...
/ [4 files][327.9 MiB/327.9 MiB]                                                
Operation completed over 4 objects/327.9 MiB.                                    


In [10]:
!gsutil cp -r gs://siliconwildcat_poc_bucket/tts_infer/hifigan-v2 /content

Copying gs://siliconwildcat_poc_bucket/tts_infer/hifigan-v2/best_model.pth.tar...
Copying gs://siliconwildcat_poc_bucket/tts_infer/hifigan-v2/config.json...
Copying gs://siliconwildcat_poc_bucket/tts_infer/hifigan-v2/scale_stats.npy...
Copying gs://siliconwildcat_poc_bucket/tts_infer/hifigan-v2/scale_stats_new.npy...
\ [4 files][820.4 MiB/820.4 MiB]   52.3 MiB/s                                   
Operation completed over 4 objects/820.4 MiB.                                    


In [11]:
synthesizer = Synthesizer(
    "/content/glowtts-v2/best_model.pth.tar",
    "/content/glowtts-v2/config.json",
    None,
    "/content/hifigan-v2/best_model.pth.tar",
    "/content/hifigan-v2/config.json",
    None,
    None,
    False,
)
symbols = synthesizer.tts_config.characters.characters

 > Using model: glow_tts


/content/TTS/TTS/tts/layers/glow_tts/glow.py:84: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  /pytorch/aten/src/ATen/native/BatchLinearAlgebra.cpp:1940.)
  w_init = torch.qr(torch.FloatTensor(self.num_splits, self.num_splits).normal_())[0]


 > Generator Model: hifigan_generator
Removing weight norm...


#음성 합성

실제 음성 합성을 수행합니다.

`long_text`의 값을 변경하여 다른 문장의 합성도 시도해보실 수 있습니다.

In [12]:
texts = """
아래 문장들은 모델 학습을 위해 사용하지 않은 문장들입니다.
서울특별시 특허허가과 허가과장 허과장.
경찰청 철창살은 외철창살이고 검찰청 철창살은 쌍철창살이다.
지향을 지양으로 오기하는 일을 지양하는 언어 습관을 지향해야 한다.
그러니까 외계인이 우리 생각을 읽고 우리 생각을 우리가 다시 생각토록 해서 그 생각이 마치 우리가 생각한 것인 것처럼 속였다는 거냐?
안 촉촉한 초코칩 나라에 살던 안 촉촉한 초코칩이 촉촉한 초코칩 나라의 촉촉한 초코칩을 보고 촉촉한 초코칩이 되고 싶어서 촉촉한 초코칩 나라에 갔는데 촉촉한 초코칩 나라의 촉촉한 문지기가 넌 촉촉한 초코칩이 아니고 안 촉촉한 초코칩이니까 안 촉촉한 초코칩 나라에서 살라고 해서 안 촉촉한 초코칩은 촉촉한 초코칩이 되는 것을 포기하고 안 촉촉한 눈물을 흘리며 안 촉촉한 초코칩 나라로 돌아갔다.
"""
for text in normalize_multiline_text(texts):
    wav = synthesizer.tts(text, None, None)
    IPython.display.display(IPython.display.Audio(wav, rate=22050))  

 > Text splitted to sentences.
['아래 문장들은 모델 학습을 위해 사용하지 않은 문장들입니다.']
 > Processing time: 1.4478936195373535
 > Real-time factor: 0.2866511125448808


 > Text splitted to sentences.
['서울특별시 특허허가과 허가과장 허과장.']
 > Processing time: 0.8406481742858887
 > Real-time factor: 0.229819136121353


 > Text splitted to sentences.
['경찰청 철창쌀은 외철창쌀이고 검찰청 철창쌀은 쌍철창쌀이다.']
 > Processing time: 1.5291731357574463
 > Real-time factor: 0.23475456474498504


 > Text splitted to sentences.
['지향을 지양으로 오기하는 일을 지양하는 언어 습관을 지향해야 한다.']
 > Processing time: 1.2120630741119385
 > Real-time factor: 0.2314499686865062


 > Text splitted to sentences.
['그러니까 외계인이 우리 생각을 읽고 우리 생각을 우리가 다시 생각토록 해서 그 생각이 마치 우리가 생각한 것인 것처럼 속였다는 거냐?']
 > Processing time: 2.5853564739227295
 > Real-time factor: 0.2346357846970538


 > Text splitted to sentences.
['안 촉촉한 초코칩 나라에 살던 안 촉촉한 초코칩이 촉촉한 초코칩 나라의 촉촉한 초코칩을 보고 촉촉한 초코칩이 되고 싶어서 촉촉한 초코칩 나라에 갔는데 촉촉한 초코칩 나라의 촉촉한 문지기가 넌 촉촉한 초코칩이 아니고 안 촉촉한 초코칩이니까 안 촉촉한 초코칩 나라에서 살라고 해서 안 촉촉한 초코칩은 촉촉한 초코칩이 되는 것을 포기하고 안 촉촉한 눈물을 흘리며 안 촉촉한 초코칩 나라로 돌아갔다.']
 > Processing time: 7.599053621292114
 > Real-time factor: 0.23333806670624974
